In [1]:
import warnings
from typing import List

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from autumn.core.inputs.database import get_input_db
from autumn.core.inputs.demography.queries import get_population_by_agegroup
from autumn.core.project import ParameterSet, get_project
from autumn.core.utils.display import pretty_print

In [2]:
warnings.filterwarnings("ignore")

## Baseline Population Population

In [3]:
input_db = get_input_db()
pop = input_db.query(table_name="population", conditions={"iso3": "KIR"})

def find_agebins(x, bins: List[int], labels: List[str]):
    return pd.cut(x, bins, labels=labels, right=False)


pop["age_bins"] = find_agebins(
    pop["start_age"], [0, 5, 15, 35, 50, 100], ["0-4", "5-14", "15-34", "35-49", "50+"]
)
pop = pop.groupby(["year", "age_bins"])["population"].agg(["sum"]).reset_index()
pop['sum'] = pop['sum'].astype('int')
pop['type'] = "Empriric"

In [4]:
fig1 = go.Figure()
fig1.update_layout(
    template = "plotly",
    xaxis =  dict(title_text="Year"),
    yaxis = dict(title_text="Population"),
    barmode = "stack",
)

for group in pop.age_bins.unique():
    plot_df = pop[pop.age_bins == group]
    fig1.add_trace(
        go.Bar(x = plot_df['year'], y = plot_df['sum'], name = group)
    )

fig1

## Running an empty model

In [5]:
project = get_project("tb_dynamics", "kiribati")

In [6]:
base_params = project.param_set.baseline

In [7]:
model_0 = project.run_baseline_model(base_params)

In [8]:
df_0 = model_0.get_outputs_df()
derived_df_0 = model_0.get_derived_outputs_df()

In [9]:
total_pop = input_db.query(table_name="population", conditions={"iso3": base_params['iso3']})
total_pop["age_bins"] = find_agebins(total_pop["start_age"], [0, 100], ["Total"])
total_pop = total_pop.groupby(["year", "age_bins"])["population"].agg(["sum"]).reset_index()

In [10]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="population_size",
)
fig2_2 = px.scatter(total_pop, x="year", y="sum")
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Actual Population", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

## Modeled age group

In [11]:
modeled_pop = df_0.filter(regex='^susceptible', axis=1).reset_index()
modeled_pop.rename(columns = {'index':'year', 'variable':'age_bins','value':'sum'}, inplace=True)
modeled_pop = modeled_pop.melt(id_vars='year')
modeled_pop['variable'] = modeled_pop['variable'].replace(['susceptibleXage_0','susceptibleXage_5','susceptibleXage_15', 'susceptibleXage_35', 'susceptibleXage_50'],
                        ['0-4','5-14','15-34','35-49', "50+"])
modeled_pop.rename(columns = {'variable':'age_bins','value':'sum'}, inplace=True)
ryear = list(range(int(base_params['time']['start']), int(base_params['time']['end'] + 1), 5))
modeled_pop = modeled_pop[modeled_pop['year'].isin(ryear)]
modeled_pop['sum'] = modeled_pop['sum'].astype('int')
modeled_pop['year'] = modeled_pop['year'].astype('int')
modeled_pop['type'] = "Modeled"
modeled_pop.reset_index(drop=True, inplace=True)

In [12]:
total_pop = pd.concat([pop,modeled_pop])

In [13]:
fig4 = go.Figure()
fig4.update_layout(
    template = "plotly",
    xaxis =  dict(title_text="Year"),
    yaxis = dict(title_text="Population"),
    barmode = "stack",
)

for group in total_pop['age_bins'].unique():
    plot_df = total_pop[total_pop['age_bins'] == group]
    fig4.add_trace(
        go.Bar(x = [plot_df['year'],plot_df['type']], y = plot_df['sum'], name = group)
    )

fig4

In [14]:
fig5 =  px.bar(
    total_pop,
    x='year',
    y='sum',
    color='age_bins',
    facet_col="type",
    labels={'year':'Year', 'sum':'Population', 'age_bins':'Age group'}
)
fig5.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig5.show()